In [5]:
%cd /scratch/bruingjde/SNAM2021-code/

import os
import typing

import joblib
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
import scipy
import seaborn as sns
import sklearn.linear_model
import sklearn.model_selection
import sklearn.metrics
import sklearn.pipeline
import sklearn.preprocessing
from tqdm.auto import tqdm

import tlp

/scratch/bruingjde/SNAM2021-code


In [18]:
# Check
ok = True
for entry in os.scandir('data'):
  if not os.path.isfile(os.path.join(entry, 'features', 'cn.pkl')):
    ok = False
    print(entry)
if ok: print('Ok!')

Ok!


In [6]:
def cn(path: str, verbose: bool = False):
  # Check if file exists
  feature_path = os.path.join(path, 'features')
  file = os.path.join(feature_path, 'cn.pkl')
  if os.path.isfile(file): return 
  
  os.makedirs(feature_path, exist_ok=True)

  # Read in
  edgelist_mature_file = os.path.join(path, 'edgelist_mature.pkl')
  if verbose: print(f'Read {edgelist_mature_file}')
  edgelist_mature = pd.read_pickle(edgelist_mature_file)
  
  instances_file = os.path.join(path, 'instances_sampled.npy')
  if verbose: print(f'Read {instances_file}')  
  instances_sampled = np.load(instances_file)

  G = nx.from_pandas_edgelist(edgelist_mature)
  
  scores = [len(list(nx.common_neighbors(G, *instance))) for instance in tqdm(instances_sampled, disable=not verbose)]
  
  result = {tlp.Experiment(feature='CN', time_aware=False): np.array(scores)}
  joblib.dump(result, os.path.join(feature_path, 'cn.pkl'))

In [15]:
cn('data/27', verbose=True)

Read data/27/edgelist_mature.pkl
Read data/27/instances_sampled.npy


  0%|          | 0/20007 [00:00<?, ?it/s]

In [ ]:
entries = sorted(os.scandir('data'), key=lambda x: x.name)
tlp.ProgressParallel(n_jobs=len(entries), total=len(entries))(
  joblib.delayed(cn)(entry.path) for entry in entries
)